# TiDB Python SDK

A powerful Python SDK for vector storage and retrieval operations with TiDB.

- 🔄 Automatic embedding generation
- 🔍 Vector similarity search
- 🎯 Advanced filtering capabilities
- 📦 Bulk operations support

## Installation

In [ ]:
%pip install autoflow-ai==0.0.1.dev29

## Configuration

- Go [tidbcloud.com](https://tidbcloud.com/) or using [tiup playground](https://docs.pingcap.com/tidb/stable/tiup-playground/) to create a free TiDB database cluster.
- Go [OpenAI platform](https://platform.openai.com/api-keys) to create your API key.

Configuration can be provided through environment variables, or using `.env`:

In [ ]:
# Create .env file, then edit your .env, for example:
# $ cat .env
# TIDB_HOST=localhost
# TIDB_PORT=4000
# TIDB_USERNAME=root
# TIDB_PASSWORD=
# OPENAI_API_KEY='your_openai_api_key'
#
# Or you can use DATABASE_URL to connect to TiDB, for example:
# $ cat .env
# DATABASE_URL=mysql+pymysql://root:@localhost:4000/test
#
# If you are using TiDB Serverless, the DATABASE_URL should be like:
# DATABASE_URL=mysql+pymysql://<USERNAME:<PASSWORD>@<HOST>:4000/test&ssl_verify_cert=true&ssl_verify_identity=true
%cp .env.example .env

In [2]:
import dotenv

dotenv.load_dotenv()

True

## Quickstart

### Connect to TiDB

In [3]:
import os
from autoflow.storage.tidb import TiDBClient

db = TiDBClient.connect(
    host=os.getenv("TIDB_HOST"),
    port=int(os.getenv("TIDB_PORT")),
    username=os.getenv("TIDB_USERNAME"),
    password=os.getenv("TIDB_PASSWORD"),
    database=os.getenv("TIDB_DATABASE"),
)

# If you are using DATABASE_URL
# db = TiDBClient.connect(database_url=os.getenv("DATABASE_URL"))

### Create table

In [4]:
from typing import Optional, Any
from autoflow.storage.tidb.schema import TableModel, Field
from autoflow.storage.tidb.embeddings import EmbeddingFunction

# Define your embedding model.
text_embed = EmbeddingFunction("openai/text-embedding-3-small")


class Chunk(TableModel, table=True):
    __tablename__ = "chunks"
    __table_args__ = {"extend_existing": True}

    id: int = Field(primary_key=True)
    text: str = Field()
    text_vec: Optional[Any] = text_embed.VectorField(
        source_field="text"
    )  # 👈 Define the vector field.
    user_id: int = Field()


table = db.create_table(schema=Chunk)

### Insert Data

🔢 Auto embedding: when you insert new data, the SDK automatically embeds the corpus for you.

In [5]:
table.truncate()
table.insert(
    Chunk(text="The quick brown fox jumps over the lazy dog", user_id=1),
)
table.bulk_insert(
    [
        Chunk(text="A quick brown dog runs in the park", user_id=2),
        Chunk(text="The lazy fox sleeps under the tree", user_id=2),
        Chunk(text="A dog and a fox play in the park", user_id=3),
    ]
)
table.rows()

4

### Vector Search

In [6]:
chunks = (
    table.search(
        "A quick fox in the park"
    )  # 👈 The query will be embedding automatically.
    .filter({"user_id": 2})
    .limit(2)
    .to_pydantic()
)
[(c.text, c.score) for c in chunks]

[('A quick brown dog runs in the park', 0.6654495199158884),
 ('The lazy fox sleeps under the tree', 0.5546760032932834)]

### Advanced Filtering

TiDB Client supports various filter operators for flexible querying:

| Operator | Description | Example |
|----------|-------------|---------|
| `$eq` | Equal to | `{"field": {"$eq": "hello"}}` |
| `$gt` | Greater than | `{"field": {"$gt": 1}}` |
| `$gte` | Greater than or equal | `{"field": {"$gte": 1}}` |
| `$lt` | Less than | `{"field": {"$lt": 1}}` |
| `$lte` | Less than or equal | `{"field": {"$lte": 1}}` |
| `$in` | In array | `{"field": {"$in": [1, 2, 3]}}` |
| `$nin` | Not in array | `{"field": {"$nin": [1, 2, 3]}}` |
| `$and` | Logical AND | `{"$and": [{"field1": 1}, {"field2": 2}]}` |
| `$or` | Logical OR | `{"$or": [{"field1": 1}, {"field2": 2}]}` |


In [7]:
chunks = table.query({"user_id": 1})
[(c.id, c.text, c.user_id) for c in chunks]

[(1, 'The quick brown fox jumps over the lazy dog', 1)]

### Multiple Tables Join

In [8]:
# Create a table to stored user data:
class User(TableModel, table=True):
    __tablename__ = "users"
    __table_args__ = {"extend_existing": True}

    id: int = Field(primary_key=True)
    name: str = Field(max_length=20)


user_table = db.create_table(schema=User)

In [9]:
user_table.truncate()
user_table.insert(User(id=1, name="Alice"))

User(id=1, name='Alice')

In [10]:
from sqlmodel import select, Session

db_engine = db.db_engine
with Session(db_engine) as db_session:
    query = (
        select(Chunk).join(User, Chunk.user_id == User.id).where(User.name == "Alice")
    )
    chunks = db_session.exec(query).all()

[(c.id, c.text, c.user_id) for c in chunks]

[(1, 'The quick brown fox jumps over the lazy dog', 1)]

### Update Data

In [11]:
old_chunk = table.query({"text": "A quick brown dog runs in the park"})[0]
chunk_id = old_chunk.id
(old_chunk.text, old_chunk.text_vec)

('A quick brown dog runs in the park',
 array([-0.0412815 , -0.00934362,  0.01239674, ..., -0.00587278,
        -0.00735941,  0.01383422], dtype=float32))

In [12]:
table.update(
    values={
        "text": "foo"  # 👈 Only provide the fields you want to update.
    },
    filters={"id": chunk_id},
)

In [13]:
new_chunk = table.get(chunk_id)
(new_chunk.text, new_chunk.text_vec)  # 👈 The vector field is updated automatically.

('foo',
 array([-0.00584331, -0.0131113 , -0.01458535, ..., -0.00907705,
        -0.0073632 , -0.01883118], dtype=float32))

### Delete Data

In [14]:
table.rows()

4

In [15]:
table.delete(filters={"user_id": 2})

In [16]:
table.rows()

2

### Execute raw SQL

- Using `execute()` to execute INSERT / UPDATE / DELETE statement.
- Using `query()` to execute SELECT / SHOW statement.

#### Execute SQL

The `execute()` method will return a `SQLExecuteResult` instance.

In [17]:
db.execute("INSERT INTO chunks(text, user_id) VALUES ('inserted from raw sql', 5)")

SQLExecuteResult(rowcount=1, success=True, message=None)

Both of the `execute` and `query` methods are support the **Parameterized SQL** feature, which help you avoid [SQL injection](https://en.wikipedia.org/wiki/SQL_injection) while using the dynamic SQL feature.

In [18]:
db.execute(
    "INSERT INTO chunks(text, user_id) VALUES (:text, :user_id)",
    {"text": "inserted from dynamic sql", "user_id": 6},
)

SQLExecuteResult(rowcount=1, success=True, message=None)

#### Query SQL

The `query()` method will return a `SQLQueryResult` instance with some helper methods:

- `to_pandas()`
- `to_list()`
- `to_rows()`
- `scalar()`

The `to_pandas()` method to convert the query result to a `pandas.DataFrame`, which is displayed as human-friendly style on the notebook:

In [19]:
db.query("SELECT id, text, user_id FROM chunks").to_pandas()

,id,text,user_id
0,1,The quick brown fox jumps over the lazy dog,1
1,4,A dog and a fox play in the park,3
2,5,inserted from raw sql,5
3,6,inserted from dynamic sql,6


The `to_list()` method will convert the query result into a list of dict.

In [20]:
db.query(
    "SELECT id, text, user_id FROM chunks WHERE user_id = :user_id", {"user_id": 3}
).to_list()

[{'id': 4, 'text': 'A dog and a fox play in the park', 'user_id': 3}]

The `to_rows` method will return a list of tuple, every tuple represent of one row of data.

In [21]:
db.query("SHOW DATABASES;").to_rows()

[('INFORMATION_SCHEMA',),
 ('METRICS_SCHEMA',),
 ('PERFORMANCE_SCHEMA',),
 ('mysql',),
 ('sys',),
 ('test',)]

In [22]:
db.query("SELECT COUNT(*) FROM chunks;").scalar()

4

### Table Management

#### List tables

In [23]:
db.table_names()

['chunks',
 'chunks_438476e0-74f2-480f-b220-573e3f663d52',
 'chunks_655b6cf3-8b30-4839-ba8b-5ed3c502f30e',
 'documents',
 'entities_438476e0-74f2-480f-b220-573e3f663d52',
 'entities_655b6cf3-8b30-4839-ba8b-5ed3c502f30e',
 'relationships_438476e0-74f2-480f-b220-573e3f663d52',
 'relationships_655b6cf3-8b30-4839-ba8b-5ed3c502f30e',
 'test_auto_embedding',
 'test_get_data',
 'test_raw_sql',
 'test_vector_search',
 'users']

#### Truncate table

Clear all data in the table:

In [24]:
table.truncate()
table.rows()

0

#### Drop table


In [25]:
db.drop_table("chunks")

SQLExecuteResult(rowcount=0, success=True, message=None)